In [ ]:
# Импорт библиотек для работы с данными
import pandas as pd
import numpy as np
import plotly.express as px
import scipy.stats as stats

# Импорт библиотек для визуализации данных
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Импорт библиотек для пред обработки данных
from sklearn.preprocessing import MaxAbsScaler, RobustScaler

# Импорт библиотек для отбора признаков
from sklearn.feature_selection import RFE, RFECV, SelectFromModel, SelectFpr, chi2

# Импорт библиотек линейной регрессии
import statsmodels.api as sm
from statsmodels.regression.recursive_ls import RecursiveLS
from statsmodels.regression.linear_model import OLS, GLS, GLSAR, WLS


# Импорт библиотек для машинного обучения
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv("input/data.csv", delimiter=";")
# Устанавливаем столбец с датой в качестве индекса DataFrame
def interpolate_data(data, method='linear', freq='M'):
    data_quarterly = data.copy()
    data_quarterly['Год'] = pd.to_datetime(data_quarterly['Год'].astype(str), format='%Y')
    data_quarterly.set_index('Год', inplace=True)
    data_quarterly = data_quarterly.resample(freq).mean().interpolate(method=method)
    return px.scatter(data_quarterly)

In [ ]:
px.scatter(data)

In [ ]:
interpolate_data(data, method='cubic')

In [ ]:
interpolate_data(data, method='quadratic')

In [ ]:
interpolate_data(data, method='akima')

In [ ]:
interpolate_data(data, method='linear')